In [13]:
#!pip -q install llama-index langchain llama-index-readers-web llama-index-embeddings-langchain llama-index-llms-huggingface langchain-community llama-index-llms-huggingface-api

# Imports and constants

In [29]:
import os
from getpass import getpass

from llama_index.core import Settings
# from llama_index.llms.huggingface import HuggingFaceInferenceAPI
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from langchain.embeddings.huggingface import HuggingFaceInferenceAPIEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, load_index_from_storage, get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.storage import StorageContext

In [15]:
# HF_TOKEN = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN
PERSIST_DIR = './storage'
MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"
EMBED_MODEL = "thenlper/gte-large"

# Storing Context
* defining the llm end embedding model
* storing the context using Indexing
* retrieving the context from the storage context

## Define the:
> Persist Directory - The persist directory in LlamaIndex is the location on disk where the indexed data and metadata are stored to avoid the time and cost of re-indexing the data.

>LLM Model - LLM Model uses the query and the retrieved documents to generate a response

>Embedding Model - Embedding Model generates vector embeddings which are to be stored in a vector store.

In [16]:
Settings.llm = HuggingFaceInferenceAPI(model_name=MODEL_NAME,
                                      token=HF_TOKEN)
Settings.embed_model = LangchainEmbedding(HuggingFaceInferenceAPIEmbeddings(model_name=EMBED_MODEL,
                                                                            api_key=HF_TOKEN))

In [26]:
if not os.path.exists(PERSIST_DIR):
    documents = SimpleDirectoryReader(input_dir="data").load_data()
    parser = SimpleNodeParser()
    nodes = parser.get_nodes_from_documents(documents)

    storage_context = StorageContext.from_defaults()
    index = VectorStoreIndex(nodes, storage_context=storage_context)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

In [30]:
query_engine = index.as_query_engine()
query_engine.query("What is the example of f1-score from the book?")

Response(response="\n\nThe example of f1-score from the book is given in the context of evaluating the performance of the proposed solution for player similarity analysis using the proposed CAE model. The authors compare the f1-score of the proposed solution with the f1-score of the previous study, which is 0.94 for the proposed solution and 0.92 for the previous study. This indicates that the proposed solution outperforms the previous study in terms of f1-score, which is a measure of the accuracy and precision of the model's predictions. The authors also provide a table (Table 7) that summarizes the top-k results and mean reciprocal rank (MRR) comparison of the proposed solution to the previous study.", source_nodes=[NodeWithScore(node=TextNode(id_='ec5b3df6-5dd3-4793-a195-b0a1408e1a7b', embedding=None, metadata={'page_label': '8', 'file_name': 'RishiAluri_Dissertation.pdf', 'file_path': '/home/rishicarter/git/GenAI/Rag and Agents/data/RishiAluri_Dissertation.pdf', 'file_type': 'appli

In [31]:
response_synthesizer = get_response_synthesizer()
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=2)

In [32]:
vector_query_engine = RetrieverQueryEngine(retriever=vector_retriever,
                                          response_synthesizer=response_synthesizer)

In [33]:
print(vector_query_engine.query("What is the example of f1-score from the book?"))



The example of f1-score from the book is given in the context of evaluating the performance of the proposed solution for player similarity analysis using the proposed CAE model. The authors compare the f1-score of the proposed solution with the f1-score of the previous study, which is 0.94 for the proposed solution and 0.92 for the previous study. This indicates that the proposed solution outperforms the previous study in terms of f1-score, which is a measure of the accuracy and precision of the model's predictions. The authors also provide a table (Table 7) that summarizes the top-k results and mean reciprocal rank (MRR) comparison of the proposed solution to the previous study.
